In [ ]:
!make clean

# Lecture 8 & 9 modern C++

* Part I: ownership and meta-programming
* Part II: more than templates

# Modern C++ part I: ownership and meta-programming

1. Pointers and ownership
    1. Raw pointer
    2. Reference
    3. Ownership
    4. Smart pointers
        1. `unique_ptr`
        2. `shared_ptr`
2. Revisit shared pointer
3. Template meta-programming and compile-time computing

# Raw pointer

The raw pointer allows us to directly manipulate the memory.  See the example file `01_pointer/01_raw_pointer.cpp`:

```cpp
struct PlainData
{
    int buffer[1024*8];
}; /* end struct PlainData */

int main(int argc, char ** argv)
{
    PlainData * ptr = nullptr;

    ptr = static_cast<PlainData *>(malloc(sizeof(PlainData)));
    std::cout << "PlainData pointer after malloc: " << ptr << std::endl;

    free(ptr);
    std::cout << "PlainData pointer after free: " << ptr << std::endl;

    ptr = new PlainData();
    std::cout << "PlainData pointer after new: " << ptr << std::endl;

    delete ptr;
    std::cout << "PlainData pointer after delete: " << ptr << std::endl;
}
```

In [ ]:
!make -C 01_pointer 01_raw_pointer ; 01_pointer/01_raw_pointer

# Reference

When we see a reference, we know that we should not deallocate / destruct the object.  See the example `01_pointer/02_reference.cpp`:

```cpp
struct PlainData
{
    int buffer[1024*8];
}; /* end struct PlainData */

// The factory function for PlainData.
PlainData * make_data()
{
    PlainData * ptr = new PlainData();
    // In a real factory function, we will do something with the allocated data
    // object before returning it.
    return ptr;
}

void manipulate_with_reference(PlainData & data)
{
    std::cout << "Manipulate with reference: " << &data << std::endl;

    for (size_t it=0; it < 1024*8; ++it)
    {
        data.buffer[it] = it;
    }
    // In a real consumer function we will do much more meaningful operations.

    // However, we cannot destruct an object passed in with a reference.
}

int main(int argc, char ** argv)
{
    PlainData * ptr = nullptr;

    ptr = make_data();
    std::cout << "PlainData pointer after factory: " << ptr << std::endl;

    manipulate_with_reference(*ptr);

    // Destruct the object where we have the pointer.
    delete ptr;
    std::cout << "PlainData pointer after delete: " << ptr << std::endl;
}
```

In [ ]:
!make -C 01_pointer 02_reference ; 01_pointer/02_reference

# Ownership

In a compliated system, memory is not free immediately after allocation.  Consider example `01_pointer/03_ownership.cpp`, where there are two worker functions with different memory management behaviors.

## Data class

Our data object is large, and we don't want the expensive overhead from frequent allocation and deallocation.

```cpp
class Data
{

public:

    constexpr const static size_t NELEM = 1024*8;

    using iterator = int *;
    using const_iterator = const int *;

    Data()
    {
        std::fill(begin(), end(), 0);
        std::cout << "Data @" << this << " is constructed" << std::endl;
    }

    ~Data()
    {
        std::cout << "Data @" << this << " is destructed" << std::endl;
    }

    const_iterator cbegin() const { return m_buffer; }
    const_iterator cend() const { return m_buffer+NELEM; }
    iterator begin() { return m_buffer; }
    iterator end() { return m_buffer+NELEM; }

    size_t size() const { return NELEM; }
    int   operator[](size_t it) const { return m_buffer[it]; }
    int & operator[](size_t it)       { return m_buffer[it]; }

    bool is_manipulated() const
    {
        for (size_t it=0; it < size(); ++it)
        {
            if ((*this)[it] != it) { return false; }
        }
        return true;
    }

private:

    // A lot of data that we don't want to reconstruct.
    int m_buffer[NELEM];

}; /* end class Data */

void manipulate_with_reference(Data & data)
{
    std::cout << "Manipulate with reference: " << &data << std::endl;

    for (size_t it=0; it < data.size(); ++it)
    {
        data[it] = it;
    }
    // In a real consumer function we will do much more meaningful operations.

    // However, we cannot destruct an object passed in with a reference.
}
```

## Separate memory management

The memory allocation and deallocation is not consistent in `worker1` and `worker2`.  This kind of problems are commonplace. 

```cpp
Data * worker1()
{
    // Create a new Data object.
    Data * data = new Data();

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

/*
 * Code in this function is intentionally made to be lack of discipline to
 * demonstrate how ownership is messed up.
 */
void worker2(Data * data)
{
    // The prerequisite for the caller to write correct code is to read the
    // code and understand when the object is alive.
    if (data->is_manipulated())
    {
        delete data;
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    Data * data = worker1();
    std::cout << "[use case 1] Data pointer after worker 1: " << data << std::endl;
    worker2(data);
    std::cout << "[use case 1] Data pointer after worker 2: " << data << std::endl;

    // You have to read the code of worker2 to know that data could be
    // destructed.  In addition, the Data class doesn't provide a
    // programmatical way to detect whether or not the object is alive.  The
    // design of Data, worker1, and worker2 makes it impossible to write
    // memory-safe code.
#ifdef CRASHME // The fenced code causes double free.
    delete data;
    std::cout << "[use case 1] Data pointer after delete: " << data << std::endl;
#endif
}
```

In [ ]:
!make -C 01_pointer 03_ownership ; 01_pointer/03_ownership

In [ ]:
# See how it crashes with double free.
!make -C 01_pointer clean
!env FLAGS=-DCRASHME make -C 01_pointer 03_ownership ; 01_pointer/03_ownership

## What is ownership

The above example shows the problem of lack of ownership.  "Ownership" isn't officially a language construct in C++, but is a common concept in many programming language for dynamic memory management.

To put it simply, when the object is "owned" by a construct or piece of code, it is assumed that it is safe for the piece of code to use that object.  The ownership assures the life of the object, and the object is not destructed when it is owned by someone.  It also means that the owner is reponsible for making sure the object gets destructed when it should be.

As we observed in the above example code, there is no way for us to let the code to know the ownership, and it is unsafe to use the `data` object after `worker2` is called.  The way C++ handles the situation is to use smart pointers.

# `std::unique_ptr`

(Modern) C++ provides two smart pointers: `unique_ptr` and `shared_ptr`.  We start with `unique_ptr` because it is lighter-weight.  A `unique_ptr` takes the same number of bytes of a raw pointer.  It may be a drop-in replace with a raw pointer.

`unique_ptr` should be used when there can only be one owner of the pointed object.

See the example code `01_pointer/04_unique.cpp`.

```cpp
static_assert(sizeof(Data *) == sizeof(std::unique_ptr<Data>), "unique_ptr should take only a word");

std::unique_ptr<Data> worker1()
{
    // Create a new Data object.
    std::unique_ptr<Data> data = std::make_unique<Data>();

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

void worker2(std::unique_ptr<Data> data)
{
    if (data->is_manipulated())
    {
        data.reset();
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    std::unique_ptr<Data> data = worker1();
    std::cout << "[use case 1] Data pointer after worker 1: " << data.get() << std::endl;

#ifdef COPYNOWORK
    worker2(data);
#else
    worker2(std::move(data));
#endif
    std::cout << "[use case 1] Data pointer after worker 2: " << data.get() << std::endl;

    data.reset();
    std::cout << "[use case 1] Data pointer after delete: " << data.get() << std::endl;
}
```

In [ ]:
!make -C 01_pointer 04_unique ; 01_pointer/04_uniquec

In [ ]:
# See how it crashes with double free.
!make -C 01_pointer clean
!env FLAGS=-DCOPYNOWORK make -C 01_pointer 04_unique ; 01_pointer/04_unique

# `std::shared_ptr`

Unlike `unique_ptr`, `shared_ptr` allows multiple owners.  It maintains a reference counter.  When the `std::shared_ptr` object is constructed, the counter increments.  When the pointer object (note, not the pointed object) is destructed, the counter decrements.  When the counter decrements from 1, the pointed object gets destructed.

`std::shared_ptr` provides `use_count()` function for showing the reference counts.  This reference counting technique is commonplace for managing ownership, and it appears in many other languages.

See the example code in `01_pointer/05_shared`.

```cpp
static_assert(sizeof(Data *) < sizeof(std::shared_ptr<Data>), "shared_ptr uses more than a word");

std::shared_ptr<Data> worker1()
{
    // Create a new Data object.
    std::shared_ptr<Data> data = std::make_shared<Data>();

    std::cout << "worker 1 data.use_count(): " << data.use_count() << std::endl;

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

void worker2(std::shared_ptr<Data> data)
{
    std::cout << "worker 2 data.use_count(): " << data.use_count() << std::endl;

    if (data->is_manipulated())
    {
        data.reset();
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = worker1();
    std::cout << "Data pointer after worker 1: " << data.get() << std::endl;

    worker2(data);
    std::cout << "Data pointer after worker 2: " << data.get() << std::endl;

    data.reset();
    std::cout << "Data pointer after reset from outside: " << data.get() << std::endl;
    std::cout << "main data.use_count(): " << data.use_count() << std::endl;
}
```

In [ ]:
!make -C 01_pointer 05_shared ; 01_pointer/05_shared

# Raw pointers vs smart pointers

The rule of thumb is to always start with smart pointers.  When in doubt, use `unique_ptr`.  `unique_ptr` forces a developer to think clearly about whether or not multiple owners are necessary.  Only use `shared_ptr` when it is absolutely necessary.  The reference counter is much more expensive than it looks.

# Modern C++ part II: more than templates

1. Copy elision / RVO, move semantics
2. Variadic template and perfect forwarding
3. Anonymous function, closure, lambda expression

# Exercises

1. Write code so that when `std::unique_ptr` is destructed, the object it points to doesn't destruct.

# References

1. TBA